In [ ]:
import sys

from utils.binaries import *
from utils.plotting import *

In [ ]:
data = np.random.normal(size=1000)

In [ ]:
plot.shaded_hist(data, cmap='coolwarm', bins=100)

In [ ]:
import logging

In [1]:
from utils import *

20:33:15 (   +4.2s) [INFO   ] -- import logging
20:33:15 (  +190ms) [INFO   ] -- import numpy as np
20:33:15 (   +19ms) [INFO   ] -- import uncertainties
20:33:16 (  +458ms) [INFO   ] -- import pandas as pd
20:33:16 (    +2ms) [INFO   ] -- import binaries.tools as tools
20:33:17 (  +873ms) [INFO   ] -- import matplotlib.pyplot as plt
20:33:17 (  +735ms) [INFO   ] -- import seaborn as so
20:33:17 (   +11ms) [DEBUG  ] -- font size set to 9.5
20:33:17 (    +5ms) [DEBUG  ] -- label size set to 13.0
20:33:17 (    +2ms) [DEBUG  ] -- figure size set to [6.6, 3.3]
20:33:17 (    +3ms) [DEBUG  ] -- markersize set to 2.0
20:33:17 (    +1ms) [DEBUG  ] -- usetex set to False
20:33:17 (    +2ms) [INFO   ] -- import plotting.tools as plot


In [ ]:
test.info('hallo')

In [2]:
Auger.SD

AttributeError: module 'utils.Auger' has no attribute 'SD'

In [4]:
Auger.__dict__

{'__name__': 'utils.Auger',
 '__doc__': None,
 '__package__': 'utils.Auger',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x7fa33077ee00>,
 '__spec__': ModuleSpec(name='utils.Auger', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7fa33077ee00>, origin='/cr/users/filip/bin/utils/Auger/__init__.py', submodule_search_locations=['/cr/users/filip/bin/utils/Auger']),
 '__path__': ['/cr/users/filip/bin/utils/Auger'],
 '__file__': '/cr/users/filip/bin/utils/Auger/__init__.py',
 '__cached__': '/cr/users/filip/bin/utils/Auger/__pycache__/__init__.cpython-310.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'),
  '__build_class__': <function 

In [5]:
from utils.Auger import SD

In [7]:
SD.__dict__

{'__name__': 'utils.Auger.SD',
 '__doc__': None,
 '__package__': 'utils.Auger.SD',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x7fa330272c20>,
 '__spec__': ModuleSpec(name='utils.Auger.SD', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7fa330272c20>, origin='/cr/users/filip/bin/utils/Auger/SD/__init__.py', submodule_search_locations=['/cr/users/filip/bin/utils/Auger/SD']),
 '__path__': ['/cr/users/filip/bin/utils/Auger/SD'],
 '__file__': '/cr/users/filip/bin/utils/Auger/SD/__init__.py',
 '__cached__': '/cr/users/filip/bin/utils/Auger/SD/__pycache__/__init__.cpython-310.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'),
  '__bu